In [ ]:
import json
import os
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
MODEL = "/kaggle/input/pp21models/A_all.fold_1"

In [ ]:
paths = glob("/kaggle/input/**/test_images/*.jpg", recursive=True)
files = list(map(os.path.basename, paths))

In [ ]:
model = tf.keras.models.load_model(f"{MODEL}.h5")
meta = json.loads(Path(f"{MODEL}.meta.json").read_text())

In [ ]:
def _path_to_x(path):
    x = tf.io.read_file(path)
    x = tf.image.decode_jpeg(x, channels=3)
    x = (
        tf.image.resize(
            x,
            tuple(meta["image_size"][:2]),
            method=tf.image.ResizeMethod.BICUBIC,
        )
        .numpy()
        .astype(np.uint8)
    )
    return x


def _y_to_labels(y):
    labels = []

    for i, v in enumerate(y):
        if v > 0.5:
            labels.append(meta["classes"][i])

    if len(labels) == 0:
        labels = ["healthy"]

    return " ".join(labels)

In [ ]:
def _predict(batch):
    X = []
    labels = []

    for path in batch:
        X.append(_path_to_x(path))

    y = model.predict(
        np.array(X),
        batch_size=meta["args"]["batch"],
        verbose=0,
    )

    return list(map(_y_to_labels, y))


labels = []
batch = []

for path in paths:
    batch.append(path)
    if len(batch) == meta["args"]["batch"]:
        labels += _predict(batch)
        batch = []

if len(batch) > 0:
    labels += _predict(batch)

In [ ]:
dfs = pd.DataFrame()
dfs.insert(0, "image", files)
dfs.insert(1, "labels", labels)
dfs.to_csv("/kaggle/working/submission.csv", index=False)

In [ ]:
# ! cat /kaggle/working/submission.csv